In [1]:
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from rich import print
from rich.panel import Panel
from rich.console import Console
from rich.markdown import Markdown
import torch

In [ ]:

base_model = "mistralai/Mistral-Nemo-Instruct-2407"
model_name = "shikhac30/mistral_nemo_12b_medical_model"  


In [3]:
# Load the model and tokenizer from HF or local path
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True
)

# Build the pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Device set to use cuda:0


In [4]:

# Input prompt
user_input = """Patient presents with acute onset of dyspnea and orthopnea;
CXR reveals bilateral perihilar infiltrates suggestive of cardiogenic pulmonary edema.
BNP elevated, echocardiogram pending to evaluate for systolic dysfunction."""

formatted_input = f"<s>[INST] {user_input.strip()} [/INST]"

result = pipe(formatted_input, max_new_tokens=128, do_sample=True, temperature=0.7)

output_text = result[0]["generated_text"].replace(formatted_input, "").strip()


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [5]:
output_text

'She has difficulty breathing and shortness of breath when lying flat. Tests show fluid buildup in her lungs, and a heart function test is planned.'

In [6]:
console = Console()
console.print(Panel.fit(user_input, title="[bold yellow]Input Prompt", border_style="green"))
console.print(Panel.fit(output_text, title="[bold magenta]Model Output", border_style="magenta"))

╭───────────────────────────────────── Input Prompt ─────────────────────────────────────╮
│ Patient presents with acute onset of dyspnea and orthopnea;                            │
│ CXR reveals bilateral perihilar infiltrates suggestive of cardiogenic pulmonary edema. │
│ BNP elevated, echocardiogram pending to evaluate for systolic dysfunction.             │
╰────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Model Output ──────────────────────────────────────────────────╮
│ She has difficulty breathing and shortness of breath when lying flat. Tests show fluid buildup in her lungs,    │
│ and a heart function test is planned.                                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [7]:
medical_notes = [

    "Patient presents with chronic obstructive pulmonary disease exacerbation, exhibiting increased sputum production, wheezing, and exertional dyspnea;\nalbuterol nebulizer and systemic corticosteroids initiated. Recommend pulmonary follow-up post-discharge.",
    "Type 2 diabetic patient with peripheral neuropathy and suboptimal glycemic control;\nrecent A1c noted at 9.2%. Commenced on insulin glargine with ongoing metformin therapy; education on dietary modifications reinforced.",
    "CT imaging reveals a 3.4 cm spiculated nodule in the left upper lobe with associated mediastinal lymphadenopathy.\nReferred to thoracic surgery for biopsy to rule out malignancy.",
    "Hypertensive urgency managed in the ED with IV labetalol;\nblood pressure reduced from 210/120 mmHg to 165/95 mmHg over 4 hours. Discharged with oral antihypertensives and close primary care follow-up.",
    "Patient with history of atrial fibrillation and recent INR of 1.4 on warfarin therapy,\npresenting with symptoms suggestive of embolic TIA. Anticoagulation strategy reassessed; bridging with LMWH considered.",

    "Postoperative day 2 s/p laparoscopic cholecystectomy,\npatient reports increasing abdominal pain and febrile episodes; ultrasound indicates possible bile leak. Interventional radiology consult obtained.",
    "Rheumatoid arthritis flare-up characterized by bilateral wrist and MCP joint swelling,\nmorning stiffness >1 hour, elevated CRP. Initiated on prednisone taper and referred to rheumatology.",
    "CHF exacerbation secondary to dietary indiscretion;\nelevated BNP, pulmonary congestion on chest x-ray. Diuresed with IV furosemide; daily weights and fluid restriction advised.",
    "Patient with ESRD on hemodialysis presenting with fever and chills;\ntunneled catheter site appears erythematous. Blood cultures drawn, empiric vancomycin initiated.",
    "Suspected sepsis secondary to UTI in elderly patient with altered mental status;\nlactate 3.8, hypotensive on presentation. Broad-spectrum antibiotics started and admitted to ICU for monitoring.",

    "Admitted for acute pancreatitis likely due to gallstones;\nlipase markedly elevated. NPO status maintained, IV fluids and analgesics administered, GI consult pending.",
    "Chronic alcohol use disorder with signs of early hepatic encephalopathy;\nelevated ammonia levels, disoriented to time. Lactulose therapy initiated and monitored for improvement.",
    "New-onset diabetes mellitus type 1 diagnosed in adolescent with polyuria, polydipsia;\nblood glucose 450 mg/dL and ketones present. Started on insulin regimen and referred to endocrinology.",
    "Advanced osteoarthritis of the right knee causing functional limitation;\nx-ray shows joint space narrowing and osteophyte formation. Referred to ortho for surgical options including TKA.",
    "60-year-old smoker with progressive dysphagia;\nEGD reveals esophageal mass. Biopsy obtained, pending pathology. Oncology referral placed for staging and treatment plan.",

    "Patient presents with chest pain radiating to the left arm;\nEKG shows ST-elevation in inferior leads. Emergent cardiac catheterization performed; stent placed in RCA.",
    "History of bipolar disorder, currently manic with pressured speech and insomnia;\nlithium levels subtherapeutic. Psychiatric evaluation done; medication titrated and patient admitted for stabilization.",
    "Middle-aged female with recurrent migraines unresponsive to triptans;\nMRI brain unremarkable. Neurology consulted for consideration of prophylactic therapy.",
    "Elderly male with history of falls;\nevaluated for gait instability, found to have vitamin B12 deficiency and positive Romberg. B12 injections initiated.",
    "Post-COVID syndrome noted with ongoing fatigue, brain fog, and palpitations;\nEKG and labs within normal limits. Referred to post-viral recovery clinic for multidisciplinary care."

]

In [ ]:
import openai
from openai import OpenAI

OPENAI_MODEL = "gpt-4.1-nano"
print(OPENAI_MODEL)
# openai.api_key = None
client = OpenAI(api_key=openai.api_key)

def gpt_simplification_score(input_text, output_text):
    eval_prompt = eval_prompt_template.format(input_text=input_text, output_text=output_text)
    eval_prompt = [
        {"role": "user", "content": eval_prompt}
    ]
    
    try:
        response = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=eval_prompt,
            max_tokens=30,
            temperature=0.6,
        )
        score_text = response.choices[0].message.content.strip()

        # score_text = response['choices'][0]['text'].strip()
        print(score_text)
        score = score_text
        # score = int(score_text.split()[0]) 
        return score
    except Exception as e:
        return f"Error: {e}"
        

gpt-4.1-nano

In [28]:
import time

console = Console()

eval_prompt_template = (
    "This is a medical simplification task.\n\n"
    "Original Input:\n{input_text}\n\n"
    "Simplified Output:\n{output_text}\n\n"
    "Rate how well the output simplifies the original input on a scale of 1 to 10,"
    "where 2 means 'not simplified at all' and 10 means 'extremely simplified'. "
    "Respond only with the number, "
)

def generate_output(pipe, prompt, max_tokens=128, temperature=0.7):
    formatted_input = f"<s>[INST] {prompt.strip()} [/INST]"
    start_time = time.time()
    result = pipe(formatted_input, max_new_tokens=max_tokens, do_sample=True, temperature=temperature)
    elapsed = time.time() - start_time
    output_text = result[0]["generated_text"].replace(formatted_input, "").strip()
    return output_text, elapsed


for idx, note in enumerate(medical_notes, start=1):
    console.rule(f"[bold green]Sample {idx}", style="green")

    output_text, elapsed = generate_output(pipe, note)

    console.print(Panel.fit(
        note,
        title="[bold yellow]Input Prompt",
        border_style="green",
        padding=(1, 2)
    ))

    console.print(Panel.fit(
        output_text,
        title="[bold magenta]Model Output",
        border_style="magenta",
        padding=(1, 2)
    ))

    gpt_score = gpt_simplification_score(note, output_text)

    
    console.print(f"[bold cyan]Generation Time:[/] {elapsed:.2f} seconds\n")
    console.print(f"[bold blue]GPT Simplification Score:[/] {gpt_score}\n")




#input text
#output text

## GPT (inpout text, output text, prompt = 'this is a medical task, 
    # and this was the input, this was output, based on these , 
    # score the text from 1- 10 , 1 being the least simplified text and 10 being highly simplified text.

──────────────────────────────────────────────────── Sample 1 ─────────────────────────────────────────────────────

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


╭───────────────────────────────────────────────── Input Prompt ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Patient presents with chronic obstructive pulmonary disease exacerbation, exhibiting increased sputum          │
│  production, wheezing, and exertional dyspnea;                                                                  │
│  albuterol nebulizer and systemic corticosteroids initiated. Recommend pulmonary follow-up post-discharge.      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Model Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  The patient has chronic lung disease and is having trouble breathing. They're getting medicine and should see  │
│  a lung doctor after leaving the hospital.                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

8

...

╭───────────────────────────────────────────── Input Prompt ──────────────────────────────────────────────╮
│                                                                                                         │
│  Post-COVID syndrome noted with ongoing fatigue, brain fog, and palpitations;                           │
│  EKG and labs within normal limits. Referred to post-viral recovery clinic for multidisciplinary care.  │
│                                                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Model Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  The patient still has some fatigue, confusion, and heart palpitations after having COVID-19. Some tests were   │
│  done and everything seems fine. They were sent to a special clinic for further care.                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

8

Generation Time: 1.31 seconds

GPT Simplification Score: 8